In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [11]:
def convert_to_sec(args):
    largs = list(args)
    return 3600 * largs[0] + 60 * largs[1] + largs[2]

In [12]:
def preprocess(place, date : str):
    # 읽기
    useful_col = [2, 11] if place == "공입" else [2, 12]
    bus_name = "5511" if place == "공입" else "관02"

    data = pd.read_csv("data\\" + bus_name + "_" + date + ".csv")
    
    # column 따두기 및 맨 윗줄 지우기
    columns = data.iloc[0, slice(2, useful_col[1])]
    columns[0] = "환산시각(초)"
    columns[-1] = "버스 환산시각(초)"
    data = data.drop(0, axis=0)

    # 시각 빈곳 채우기
    for i in range(3):
        for j in range(data.shape[0]):
            if data.iloc[j, i] is np.NaN:
                data.iloc[j, i] = data.iloc[j - 1, i]

    # 아랫쪽 빈곳 지우기
    key = [row + 1 for row in range(data.shape[0]) if all(data.iloc[row, slice(0, useful_col[1] - 4)].isnull())]
    data = data.drop(key, axis=0)
    
    # 시간 변환하기 및 column 세팅하기
    for i in range(data.shape[0]):
        data.iloc[i, 2] = convert_to_sec(map(int, data.iloc[i, 0:3]))
        if all(data.iloc[i, slice(useful_col[1] - 1, useful_col[1] + 2)].notna()):
            data.iloc[i, useful_col[1] - 1] = convert_to_sec(map(int, data.iloc[i, slice(useful_col[1] - 1, useful_col[1] + 2)]))
    data = data.iloc[:, slice(2, useful_col[1])]
    data.columns = columns
    data = data.drop(["버스 번호"], axis=1)

    # 남아 있는 사람 계산하기
    for i in range(1, data.shape[0]):
        tmp = 0
        for j in range(0, useful_col[1] - 6):
            if type(data.iloc[i, j + 1]) == type(1):
                if j == 0: tmp += data.iloc[i, j + 1]
                else: tmp -=data.iloc[i, j + 1]

        data.iloc[i, -2] = data.iloc[i - 1, -2] + tmp

    return data

In [13]:
def add_past_data():
    num_list = []
    data = pd.read_csv("data\\5511_시간대_탈사람_남은사람_시간별 구분.csv", index_col=[0])
    for i in range(0, data.shape[1], 3):
        for j in range(data.shape[0]):
            if all(data.iloc[j, i:i + 3].notna()):
                num_list.append(int(data.iloc[j, i + 1]) - int(data.iloc[j, i + 2]))

    return num_list

In [14]:
import scipy
import scipy.stats 

def apply_test(data_name, data):
    dist_names = ["norm", "gamma", "beta", "expon", "t", "chi2"]
    dist_results = []
    params = {}
    for dist_name in dist_names:
        dist = getattr(scipy.stats, dist_name)  # 분포의 특징을 가져와서
        param = dist.fit(data) # 관측값을 이용하여 가정한 분포의 모수를 추정 
        params[dist_name] = param # 추정한 모수를 Dictionary에 넣고
        Stat, p = scipy.stats.kstest(data, dist_name, param) # Kolmogorov-Smirnov 검정을 합니다. 
        dist_results.append((dist_name, p)) # 결과를 list에 넣고요
    
    print(data_name + "의 확률검정")
    print(dist_results) # 각 분포의 p value를 한 번 보고요. 
    best_dist, best_p = (max(dist_results, key=lambda item: item[1])) # 그중에 제일 p값이 큰 것을 찾고요
    print("제일 비슷한 분포: %s"%(best_dist))
    print("그 분포의 p_value: %f\n"%(best_p))

In [15]:
def draw(name, place, data):
    plt.figure()
    sns.kdeplot(data)
    plt.savefig("graphs\\" + place + "_" + name[0] + "_" + name[1] + ".png")

In [16]:
def draw_and_test(place, data):
    buses = ["온 사람", "5511", "5516", "차고지", "이탈"] if place == "공입" else ["온 사람", "관02", "5516", "5513", "셔틀", "이탈"]

    # 버스 소요 시간
    # bus_time = []
    # for i in range(data.shape[0]):
    #     if data.iloc[i, -1] is not np.nan:
    #         bus_time.append(int(data.iloc[i, -1] - data.iloc[i, 0]))

    # apply_test("버스 소요 시간_" + place, bus_time)
    # draw(["버스 소요 시간", buses[1]], place, bus_time)
    

    # 버스(유입, 이탈 포함) 도착 주기 및 인원
    for k in range(len(buses)):
        arrive_period = []
        depart_ratio = []
        tmp = 99999
        for i in range(data.shape[0]):
            if data.iloc[i, :].notna()[k + 1]:
                depart_ratio.append(float(data.iloc[i, k + 1]))
                if tmp > int(data.iloc[i, 0]):
                    tmp = int(data.iloc[i, 0])
                else:
                    for _ in range(depart_ratio[-1]):
                        arrive_period.append(int(data.iloc[i, 0]) - tmp)
                    tmp = int(data.iloc[i, 0])
        
        apply_test("도착 주기_" + place + "_" + buses[k], arrive_period)
        draw(["도착 주기", buses[k]], place, arrive_period)
        apply_test("인원수 분포_" + place + "_" + buses[k], depart_ratio)
        draw(["인원수 분포", buses[k]], place, depart_ratio)

In [17]:
place = "공입"

In [18]:
dates = ["1122", "1123", "1128"]
data_dict = dict([(date, 0) for date in dates])
for date in dates:
    data_dict[date] = preprocess(place, date)
    
# draw_and_test(pd.concat(data_dict.values()))